![IBGE LOGO](https://www.infoescola.com/wp-content/uploads/2008/02/IBGE.png)



## API IBGE Noticias


In [0]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
import pytz
from pyspark.sql.functions import lit
from pyspark.sql import Window
from pyspark.sql.functions import *
from datetime import datetime

import requests
import json

In [0]:
class RawLayerIngestion:

  def __init__(self,url,first_page,raw_directory):
    self.URL_API = url
    self.first_page = first_page
    self.raw_directory = raw_directory

  def raw_ingestion(self):
    try:
      ## Verifica a pasta data mais recente ###
      lst_folder_date = [lst_date.name.replace('/','') for lst_date in dbutils.fs.ls(f'{self.raw_directory}')]
      lst_folder_date.sort()
      ### Verifica a pasta de ingestao da API de dados mais recente  ###
      valor_maximo_raw =[int(i.name.split("_")[3]) for i in dbutils.fs.ls(f'{self.raw_directory}{lst_folder_date[-1]}/')]
      valor_maximo_raw.sort()
      valor_maximo_raw = valor_maximo_raw[-1]
      ## Verificando numero total de paginas ##
      content_json = requests.get("{url}?page={page}".format(url=self.URL_API,page=self.first_page))
      content_json = content_json.json()
      print("Ultima pagina inserida na RAw ==>  " ,valor_maximo_raw)
      acumulo_paginas = []
      np_acumulo = []
      contador = valor_maximo_raw
      if valor_maximo_raw <= content_json["totalPages"]:

        print(f"Iniciando Processo a partir da pagina ==> {contador}")
        while contador <= content_json["totalPages"]:
          noticias_API = requests.get("{url}?page={contador}".format(url=self.URL_API,contador=contador))
          noticias_json = noticias_API.json()
          acumulo_paginas.append(noticias_json["items"])
          np_acumulo.append(noticias_json['page'])
          ##### Data e hora de ingestao na camada RAW   ####
          time_file = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y%m%d_%H%M%S')

          if noticias_json["items"] != [] and str(noticias_json["page"])[-1] == "0":

            result_acumulo_paginas =[acumulo_paginas[i][item] for i in range(0,len(acumulo_paginas))
                                                                for item in range(0,len(acumulo_paginas[i]))]   
            df = spark.createDataFrame(result_acumulo_paginas).withColumn(f'PAGE',lit(f"{np_acumulo[0]} - to - {np_acumulo[-1]} - {time_file}"))
            print(f"\t Gravando até a pagina {contador}  no diretorio dbfs {self.raw_directory}{time_file[0:6]}/")
            df.write.mode("overwrite").json(f'{self.raw_directory}{time_file[0:6]}/ibgeapipage_{np_acumulo[0]}_to_{np_acumulo[-1]}_{time_file}')
            acumulo_paginas = []
            np_acumulo = []
          elif  noticias_json["page"] == content_json["totalPages"]:
            ### Ajuntando todas as paginas appendadas em uma lista unica ####
            result_acumulo_paginas =[acumulo_paginas[i][item] for i in range(0,len(acumulo_paginas))
                                                                    for item in range(0,len(acumulo_paginas[i]))]  
            df = spark.createDataFrame(result_acumulo_paginas).withColumn(f'PAGE',lit(f"{np_acumulo[0]} - to - {np_acumulo[-1]} - {time_file}"))
            print(f"\t Gravando a pagina {contador}  no diretorio dbfs {self.raw_directory}{time_file[0:6]}/")
            df.write.mode("overwrite").json(f'{self.raw_directory}{time_file[0:6]}/ibgeapipage_{np_acumulo[0]}_to_{np_acumulo[-1]}_{time_file}')
            acumulo_paginas = []
            np_acumulo = []

          contador = contador +1
        print("Processo finalizado")
      else:
        print("Todas as paginas da API ja foram inseridas na camada RAW")

    except:
      try:
        print("Primeira ingestão ou Nao possui arquivos na Data corrente")
        print("Buscando noticias desde a pagina ....")
        noticias_API = requests.get("{url}?page={page}".format(url=self.URL_API,page=self.first_page))
        noticias_json = noticias_API.json()
        acumulo_paginas = []
        np_acumulo = []

        try:
                ## Verifica a pasta data mais recente ###
          lst_folder_date = [lst_date.name.replace('/','') for lst_date in dbutils.fs.ls(f'{self.raw_directory}')]
          lst_folder_date.sort()
          ### Verifica a pasta de ingestao da API de dados mais recente para adicionar ao contador  ###
          contador =[int(i.name.split("_")[3]) for i in dbutils.fs.ls(f'{self.raw_directory}{lst_folder_date[-1]}/')]
          contador.sort()
          contador = contador[-1]
        except:
          contador = 1

        while contador <= noticias_json["totalPages"]:
          noticias_API = requests.get("{url}?page={contador}".format(url=self.URL_API,contador=contador))
          noticias_json = noticias_API.json()
          acumulo_paginas.append(noticias_json["items"])
          np_acumulo.append(noticias_json['page'])
          ##### Data e hora de ingestao na camada RAW   ####
          time_file = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y%m%d_%H%M%S')

          if noticias_json["items"] != [] and str(noticias_json["page"])[-1] == "0":

            result_acumulo_paginas =[acumulo_paginas[i][item] for i in range(0,len(acumulo_paginas))
                                                                for item in range(0,len(acumulo_paginas[i]))]   
            
            df = spark.createDataFrame(result_acumulo_paginas).withColumn(f'PAGE',lit(f"{np_acumulo[0]} - to - {np_acumulo[-1]} - {time_file}"))
            print(f"\t Gravando até a pagina {contador}  no diretorio dbfs {self.raw_directory}{time_file[0:6]}/")
            df.write.mode("overwrite").json(f'{self.raw_directory}{time_file[0:6]}/ibgeapipage_{np_acumulo[0]}_to_{np_acumulo[-1]}_{time_file}')
            acumulo_paginas = []
            np_acumulo = []
          elif noticias_json["page"] == noticias_json["totalPages"]:
            print(f"Gravando todas as paginas até {contador}  no diretorio dbfs {self.raw_directory}{time_file[0:6]}/")
            ### Ajuntando todas as paginas appendadas em uma lista unica ####
            result_acumulo_paginas =[acumulo_paginas[i][item] for i in range(0,len(acumulo_paginas))
                                                                  for item in range(0,len(acumulo_paginas[i]))]  
            
            df = spark.createDataFrame(result_acumulo_paginas).withColumn(f'PAGE',lit(f"{np_acumulo[0]} - to - {np_acumulo[-1]} - {time_file}"))
            df.write.mode("overwrite").json(f'{self.raw_directory}{time_file[0:6]}/ibgeapipage_{np_acumulo[0]}_to_{np_acumulo[-1]}_{time_file}')
            acumulo_paginas = []
            np_acumulo = []
          contador = contador +1
        print("Processo finalizado")
      except Exception as e:
        print(f"===>>>> {e}")

  def start_run(self):
    self.raw_ingestion()

In [0]:
ingestao_raw = RawLayerIngestion('http://servicodados.ibge.gov.br/api/v3/noticias/',1,'dbfs:/mnt/raw_3/')
ingestao_raw.start_run()

Ultima pagina inserida na RAw ==>   206
Iniciando Processo a partit da pagina ==> 206
	 Gravando a pagina 207  no diretorio dbfs dbfs:/mnt/raw_3/202411/
Processo finalizado


In [0]:
# df2 = spark.read.json('dbfs:/mnt/raw_3/202411/*')

# df2.display()


PAGE,data_publicacao,destaque,editorias,id,imagens,introducao,link,produto_id,produtos,produtos_relacionados,tipo,titulo
206 - to - 206 - 20241107_174333,13/02/2004 07:31:00,false,,12734,,,http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/12734-asi-agroindustria-cresce-16-em-2003-e-a-industria-em-geral-tem-crescimento-de-03.html,0,,,Release,"Agroindústria cresce 1,6% em 2003 e a indústria em geral tem crescimento de 0,3%"
206 - to - 206 - 20241107_174333,13/02/2004 07:31:00,false,,12735,,,http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/12735-asi-em-2003-industria-cresce-em-seis-das-12-regioes-pesquisadas.html,0,,,Release,"Em 2003, indústria cresce em seis das 12 regiões pesquisadas"
206 - to - 206 - 20241107_174333,13/02/2004 07:31:00,false,,12736,,,http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/12736-asi-ipca-de-janeiro-teve-variacao-de-076.html,0,,,Release,"IPCA de janeiro teve variação de 0,76%"
206 - to - 206 - 20241107_174333,06/02/2004 07:31:00,false,,14402,,"A produção industrial em dezembro do ano passado apresentou queda de 1,0% frente ao mês de novembro,...",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/14402-asi-em-dezembro-producao-industrial-cai-10-ante-novembro-e-cresce-em-todas-as-demais-comparacoes.html,0,,,Release,"Em dezembro, produção industrial cai 1,0% ante novembro e cresce em todas as demais comparações"
206 - to - 206 - 20241107_174333,16/01/2004 07:01:00,false,,14399,,"O Índice Nacional da Construção Civil do mês de dezembro, calculado pelo IBGE em convênio com a CAIXA, foi de 0,36%,",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/14399-asi-custo-nacional-da-construcao-civil-subiu-1431-em-2003.html,0,,,Release,"Custo Nacional da construção civil subiu 14,31% em 2003"
206 - to - 206 - 20241107_174333,16/01/2004 07:01:00,false,,14401,,"Em novembro de 2003, a produção industrial caiu, frente a novembro de 2002, em sete das 12 áreas pesquisadas pelo IBGE.",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/14401-asi-producao-industrial-cai-em-7-das-12-regioes-pesquisadas-em-novembro.html,0,,,Release,"Produção industrial cai em 7 das 12 regiões pesquisadas, em novembro"
206 - to - 206 - 20241107_174342,13/02/2004 07:31:00,false,,12734,,,http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/12734-asi-agroindustria-cresce-16-em-2003-e-a-industria-em-geral-tem-crescimento-de-03.html,0,,,Release,"Agroindústria cresce 1,6% em 2003 e a indústria em geral tem crescimento de 0,3%"
206 - to - 206 - 20241107_174342,13/02/2004 07:31:00,false,,12735,,,http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/12735-asi-em-2003-industria-cresce-em-seis-das-12-regioes-pesquisadas.html,0,,,Release,"Em 2003, indústria cresce em seis das 12 regiões pesquisadas"
206 - to - 206 - 20241107_174342,13/02/2004 07:31:00,false,,12736,,,http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/12736-asi-ipca-de-janeiro-teve-variacao-de-076.html,0,,,Release,"IPCA de janeiro teve variação de 0,76%"
206 - to - 206 - 20241107_174342,06/02/2004 07:31:00,false,,14402,,"A produção industrial em dezembro do ano passado apresentou queda de 1,0% frente ao mês de novembro,...",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/14402-asi-em-dezembro-producao-industrial-cai-10-ante-novembro-e-cresce-em-todas-as-demais-comparacoes.html,0,,,Release,"Em dezembro, produção industrial cai 1,0% ante novembro e cresce em todas as demais comparações"


In [0]:
df2.count()

6186